# Group Proposal

#### **Authors:** Linda Huang, Manav Kakar, Omer Tahir, Yuchen Zhang
#### **Group:** 2

## Introduction

1. **Relevant background information** on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal.

2. **Clearly state the question** you will try to answer with your project. Your question should involve one random variable of interest (the response) and one or more explanatory variables. Of the response variable, explain whether your project is focused on prediction, inference, or both.

3. **Identify and describe the dataset** that will be used to answer the question. Remember, this dataset is allowed to contain more variables than you need, in fact, exploring how the different variables in the dataset affect your model, is a crucial part of the project.

4. Also, be sure to **frame your question/objectives in terms of what is already known in the literature**. Be sure to include at least **two scientific publications** that can help frame your study (you will need to include these in the References section).

## Preliminary Results

## Methods: Plan

## References